In [77]:
# import the libraries
import requests
import csv
from bs4 import BeautifulSoup

In [95]:
# convert model names to brands
import csv

# open the models file and use that to replace model names with brand names. 
# format of file is brand, model
file = open('models.csv', 'r')
csv_file = csv.reader(file)

# don't store brand names that are weird from the csv file. like 'problems' and 'car' those arent brand names
weird_brands = ['car', 'hyndai kia', 'hyundai,', 'hyundai.', 'kia,', 'kia.', 
                'mercedes benz', 'mercedes-benz', 'nissan.', 'problem', 'seat', 'toyata','volkwagen' ]

# store the brand names
brands = []
distinct_brands = []
for i in csv_file:
    if(i[0] in weird_brands):
        continue
    else:
        brands.append(i)
        if(i[0] not in distinct_brands):
            distinct_brands.append(i[0])
#     parsedData = parsedData.replace(i[1], i[0])

print('Finished storing this csv file into a list')
# close the file
file.close()

Finished storing this csv file into a list


In [82]:
brands

[['acura', 'integra'],
 ['acura', 'Legend\xa0'],
 ['acura', 'vigor'],
 ['acura', 'rlx'],
 ['acura', 'ILX'],
 ['acura', 'MDX'],
 ['acura', 'RDX'],
 ['acura', 'TSX'],
 ['acura', 'RSX'],
 ['audi', 'quattro'],
 ['audi', 'a4'],
 ['audi', 'a6'],
 ['audi', 'a5'],
 ['audi', 'a7'],
 ['audi', 'q3'],
 ['audi', 'q5'],
 ['audi', 'q7'],
 ['audi', 'a8'],
 ['audi', 'a3'],
 ['bmw', '3-series'],
 ['bmw', '5-series'],
 ['bmw', '6-series'],
 ['bmw', '4-series'],
 ['bmw', 'xdrive'],
 ['bmw', 'alpina'],
 ['bmw', '330ci'],
 ['bmw', '328i'],
 ['bmw', '330c'],
 ['bmw', '335i'],
 ['bmw', '335d'],
 ['bmw', '528i'],
 ['bmw', '535i'],
 ['bmw', '128i'],
 ['bmw', '640i'],
 ['bmw', '3er'],
 ['bmw', '5er'],
 ['bmw', 'M5'],
 ['bmw', 'M3'],
 ['bmw', 'M6'],
 ['bmw', 'bmws'],
 ['bmw', 'bmwesque'],
 ['buick', 'park avenue'],
 ['buick', 'la crosse'],
 ['buick', 'lacrosse'],
 ['buick', 'century'],
 ['buick', 'enclave'],
 ['buick', 'encore'],
 ['buick', 'regal'],
 ['buick', 'buicks'],
 ['cadillac', 'escalade'],
 ['cadillac', 

## Scraping all pages

In [79]:
# we'll be using the cars and conversations forum
url = 'https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations'

In [94]:
# number of forum pages
num_pages = 2575

In [ ]:
# store the comments in a string
all_comments = ""

# store output: date, userid, message
output_csv = []

# go through each page
for page_number in range(1, num_pages + 1):
    # first page string of url isn't anything special
    try:
        if(page_number == 1):
            new_url = url
        else:
            # end url format is /p220 etc.
            new_url = url + "/p" + str(page_number)
            
        print(new_url)
        # collect the url with requests library
        page = requests.get(new_url)
        
        
        # get the html of the page in string form
        page_html = page.text
        
        # create the BeautifulSoup object that takes in the html in str form and a html/xml parser of choice either html.parser or lxml
        soup = BeautifulSoup(page_html, 'html.parser')
        
        # inspect the elements from the webpage to know where stuff is
        # in this case the comments are under a <p> tag which are all under a <div class = "Message userContent"</div/ tag 
        # which are all under a <div class = "MessageList DataList Comments"> tag
    
        # Pull all text from the 'MessageList DataList Comments' divs
        messagelist = soup.find(class_ = "MessageList DataList Comments")
        
        # within the 'MessageList DataList Comments' divs pull all text from 'Message userContent' divs
        #usermessages = messagelist.find_all(class_ =  "Message userContent")
        usermessages = messagelist.find_all(class_ = "Comment")
        
        # now extract just the <p> tags from all comments! 
        for i in range(len(usermessages)):
            # let's grab the date and username of the post which is under class = "Item-Header CommentHeader"
            # in a <time> tag and class = "Username"
            
            commentheader = usermessages[i].find(class_ = 'Item-Header CommentHeader')
            
            # grab the date. Format: June 18, 2014  3:53PM is stored in title = ' ' of <time  >
            date_of_post = commentheader.find("time", title = True)
            
            # variable to store date
            date_of_post = date_of_post['title']
            
            # grab the username
            username = commentheader.find(class_ = "Username")
            # variable to store username
            username = username.string
            
            # now grab text under Message userContent
            messageheader = usermessages[i].find(class_ = 'Message userContent')
            
            # a user comment might have multiple <p> tags
            # [s.extract() for s in usermessages[i](class_ = 'UserQuote')]
            [s.extract() for s in messageheader(class_ = 'UserQuote')]
            [s.extract() for s in messageheader('a')]
            [s.extract() for s in messageheader('img')]
            [s.extract() for s in messageheader('br')]
            [s.extract() for s in messageheader('blockquote')]
            
#             user_comments = messageheader.find_all('p')
            
            
            # variable to store user message
            message = messageheader.prettify()
            
            
#             for j in range(len(user_comments)):
                # remove <a>, <img>, <br> tags embedded in <p>
#                 [s.extract() for s in user_comments[j]('a')]
#                 [s.extract() for s in user_comments[j]('img')]
#                 [s.extract() for s in user_comments[j]('br')]

                
                # Let's do some string manipulation 
                # remove the newline characters, '<p>', '</p','said:'  
                # add the users comments to the mega string

#                 message += user_comments[j].prettify()
                
            # let's do some string manipulation
            # remove the newline characters, '<p>', '</p','said:'
            message = message.replace('\n', '')
            message = message.replace(r"<p>", "")
            message = message.replace(r"</p>", "")
            message = message.replace(r"</p>", "")
            message = message.replace(r"said:", "")
            message = message.replace(r":", "")
            message = message.replace(r'</div>' , "")
            message = message.lower()
            message = message.replace(r'<div class="message usercontent">' , "")
            message = message.replace(r'&gt;' , "")
            
            
            # deal with brand name
            for k in brands:
                message = message.replace(k[1], k[0])
            
#             print(message)
#             print('\n post' + ' ' + str(i) + ' \n')
            # add to all_comments
            all_comments += message
            
            # create a tuple with Date, Userid, Message and store into output_csv
            row = date_of_post, username, message
            output_csv.append(row)
            
            # print this if you want to see some output
            # print('username: ' + str(username))
            # print('date: ' + str(date_of_post))
            # print('message: ' + message)
            
        # print some checks
        print('Page ' + str(page_number) + ' complete')
    except:
        print('page ' + str(page_number) + ' failed')
        continue

https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations
Page 1 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2
Page 2 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p3
Page 3 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p4
Page 4 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p5
Page 5 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p6
Page 6 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p7
Page 7 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p8
Page 8 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p9
Page 9 complete
https://forums.edmunds.com/disc

In [93]:
output_csv[:]

[('September  6, 2017 11:22AM',
  'oldfarmer50',
  ' was watching the news where cars were lined up for miles getting gas. i supose those could be people running on empty but if not why not just drive north to an area with less panic?'),
 ('September  6, 2017 11:37AM',
  'oldfarmer50',
  ' those mini cars make sense for urban driving conditions. too bad low gas prices make them a tough sell. last year a local mitsu dealer must have bought up every year end mirage and was selling them for $9,999. had about a hundred in stock.'),
 ('September  6, 2017 11:55AM',
  'houdini1',
  ' ditto, and get an early start up to your brothers place.  if it was me, i would already be gone.'),
 ('September  6, 2017 12:35PM',
  'imidazol97',
  " be safe, . leave. the storm is predicted to be a level 3 when it comes ashore despite the media's hyping the idea that it's now a 5 and will remain a 5. also the media likes to present the pictures with the big red areas, which i've learned is the temperatures at 

## Throw output_csv list into a csv file and place all_text into a text file

In [54]:
# output into csv file
# throw this into a text file just in case
with open('finalcomments.csv','w', encoding = 'utf-8', newline = '') as out:
    csv_out = csv.writer(out)
    csv_out.writerow (['date','username', 'comment'])
    for row in output_csv:
        csv_out.writerow(row)

print('write csv is complete')

write csv is complete


In [53]:
# throw this into a text file just in case
new_file = open('alltext.txt', 'w', encoding = 'utf-8')
new_file.write(all_comments)
new_file.close()

## Word Frequency

In [55]:
# grab word frequency using nltk library
import nltk
# stop words
from nltk.corpus import stopwords
nltk.download()
nltk.download('stopwords')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [56]:
# tokenize the text 
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(all_comments)

In [57]:
# let's do a word count
from collections import Counter

# counter object
words = Counter()

# update counter with new words
words.update(tokens)

In [58]:
# check
words.most_common()[:100]

[('the', 276767),
 ('i', 182354),
 ('a', 162449),
 ('to', 150487),
 ('and', 127021),
 ('it', 103459),
 ('of', 92320),
 ('that', 84958),
 ('in', 84704),
 ('is', 69450),
 ('you', 65228),
 ('for', 62533),
 ('on', 51593),
 ('t', 48458),
 ('was', 47164),
 ('my', 46867),
 ('have', 43587),
 ('s', 43181),
 ('they', 42414),
 ('with', 42065),
 ('but', 40173),
 ('be', 33714),
 ('car', 33557),
 ('are', 31604),
 ('not', 30565),
 ('at', 29244),
 ('if', 29101),
 ('as', 28560),
 ('or', 26616),
 ('class', 25967),
 ('so', 24491),
 ('one', 24068),
 ('this', 23892),
 ('like', 23564),
 ('me', 23484),
 ('he', 22769),
 ('just', 22721),
 ('would', 22025),
 ('we', 21657),
 ('about', 21533),
 ('had', 21434),
 ('blockquote', 21185),
 ('can', 20546),
 ('up', 20436),
 ('all', 20018),
 ('out', 20011),
 ('get', 19656),
 ('from', 19477),
 ('when', 19442),
 ('an', 19134),
 ('what', 18767),
 ('there', 18722),
 ('your', 16991),
 ('more', 16435),
 ('do', 16159),
 ('will', 16153),
 ('don', 15858),
 ('has', 14551),
 ('no',

So these are the most common words, but they are also the same in any english text. Therefore they are not very insightful. Hence, let's remove single characters, numbers, and common words.

In [59]:
# store the stop words
stopwords = stopwords.words('english')[:]

In [74]:
# lets do this again: remove stop words, single letter variable names
words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1)

In [75]:
# check
words.most_common()[:1000]

[('car', 33557),
 ('class', 25967),
 ('one', 24068),
 ('like', 23564),
 ('would', 22025),
 ('blockquote', 21185),
 ('get', 19656),
 ('new', 13817),
 ('div', 13697),
 ('think', 13671),
 ('good', 13052),
 ('time', 12844),
 ('back', 11597),
 ('cars', 11445),
 ('quote', 11354),
 ('know', 10934),
 ('years', 10390),
 ('could', 9725),
 ('much', 9566),
 ('go', 9456),
 ('got', 9346),
 ('even', 9302),
 ('people', 9029),
 ('really', 8686),
 ('see', 8681),
 ('well', 8572),
 ('going', 8455),
 ('way', 8315),
 ('said', 8195),
 ('want', 7790),
 ('year', 7725),
 ('still', 7498),
 ('right', 7345),
 ('make', 7139),
 ('span', 6605),
 ('dealer', 6547),
 ('never', 6532),
 ('lot', 6471),
 ('old', 6408),
 ('also', 6321),
 ('take', 6253),
 ('better', 6226),
 ('drive', 6167),
 ('sure', 6160),
 ('last', 5939),
 ('first', 5933),
 ('buy', 5905),
 ('something', 5879),
 ('used', 5841),
 ('many', 5788),
 ('around', 5623),
 ('day', 5599),
 ('ford', 5596),
 ('probably', 5584),
 ('miles', 5511),
 ('us', 5392),
 ('say', 

In [71]:
# now let's do this again but with only brand names
words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1 and x in distinct_brands)

In [70]:
distinct_brands

['acura',
 'audi',
 'bmw',
 'buick',
 'cadillac',
 'chevrolet',
 'chrysler',
 'dodge',
 'ford',
 'honda',
 'hyundai',
 'infiniti',
 'kia',
 'lincoln',
 'mazda',
 'mercedes',
 'mercury',
 'mitsubishi',
 'nissan',
 'pontiac',
 'saturn',
 'sedan',
 'subaru',
 'suzuki',
 'toyota',
 'volkswagen',
 'volvo']

In [72]:
# check
words.most_common()[:]

[('ford', 5596),
 ('bmw', 3741),
 ('chevrolet', 3467),
 ('volkswagen', 3215),
 ('honda', 2988),
 ('mercedes', 2880),
 ('toyota', 2700),
 ('cadillac', 2679),
 ('audi', 2637),
 ('buick', 2116),
 ('nissan', 1869),
 ('chrysler', 1407),
 ('dodge', 1237),
 ('sedan', 1126),
 ('kia', 1081),
 ('hyundai', 1039),
 ('lincoln', 1028),
 ('subaru', 1022),
 ('mazda', 762),
 ('acura', 706),
 ('volvo', 652),
 ('pontiac', 427),
 ('infiniti', 410),
 ('mitsubishi', 316),
 ('mercury', 238),
 ('saturn', 70),
 ('suzuki', 35)]